In [1]:
import numpy
import timeit
import theano
import theano.tensor as T
import os
try:
    import PIL.Image as Image
except ImportError:
    import Image
    
from theano.tensor.shared_randomstreams import RandomStreams

import sys
sys.path.append("../lib")
from load import mnist
from load import faces
from rbm import RBM
from utils import tile_raster_images

Using gpu device 0: GeForce GTX 980 (CNMeM is disabled, CuDNN 3007)


In [2]:
trX, teX, trY, teY  = faces(onehot=False)
#trX, teX, trY, teY  = mnist(onehot=False)


In [3]:
train_set_x = theano.shared(numpy.asarray(trX, dtype=theano.config.floatX),borrow=True)
test_set_x = theano.shared(numpy.asarray(teX, dtype=theano.config.floatX),borrow=True)
train_set_y = theano.shared(numpy.asarray(trY, dtype="int32"),borrow=True)
test_set_y = theano.shared(numpy.asarray(teY, dtype="int32"),borrow=True)

datasets = [(train_set_x, train_set_y),(test_set_x, test_set_y)]

In [4]:
def test_rbm(dataset,learning_rate=0.005, training_epochs=10,
             batch_size=14,
             n_chains=14, n_samples=7, output_folder='rbm_plots_FER',
             n_hidden=1400):
    """
    Demonstrate how to train and afterwards sample from it using Theano.

    This is demonstrated on MNIST.

    :param learning_rate: learning rate used for training the RBM

    :param training_epochs: number of epochs used for training

    :param dataset: path the the pickled dataset

    :param batch_size: size of a batch used to train the RBM

    :param n_chains: number of parallel Gibbs chains to be used for sampling

    :param n_samples: number of samples to plot for each chain

    """

    
    train_set_x = dataset[0][0]
    test_set_x = dataset[1][0]
    train_set_y = dataset[0][1]
    test_set_y = dataset[1][1]
    
    print train_set_x.shape

    # compute number of minibatches for training, validation and testing
    n_train_batches = train_set_x.get_value(borrow=True).shape[0] // batch_size
    print str(n_train_batches)

    # allocate symbolic variables for the data
    index = T.lscalar()    # index to a [mini]batch
    x = T.matrix('x')  # the data is presented as rasterized images

    rng = numpy.random.RandomState(123)
    theano_rng = RandomStreams(rng.randint(2 ** 30))

    # initialize storage for the persistent chain (state = hidden
    # layer of chain)
    persistent_chain = theano.shared(numpy.zeros((batch_size, n_hidden),
                                                 dtype=theano.config.floatX),
                                     borrow=True)

    # construct the RBM class
    rbm = RBM(input=x, n_visible=48 * 48,
              n_hidden=n_hidden, numpy_rng=rng, theano_rng=theano_rng)

    # get the cost and the gradient corresponding to one step of CD-15
    cost, updates = rbm.get_cost_updates(lr=learning_rate,
                                         persistent=persistent_chain, k=15)

    #################################
    #     Training the RBM          #
    #################################
    if not os.path.isdir(output_folder):
        os.makedirs(output_folder)
    os.chdir(output_folder)

    # start-snippet-5
    # it is ok for a theano function to have no output
    # the purpose of train_rbm is solely to update the RBM parameters
    print str(index * batch_size)
    
    train_rbm = theano.function(
        [index],
        cost,
        updates=updates,
        givens={
            x: train_set_x[index * batch_size: (index + 1) * batch_size]
        },
        name='train_rbm'
    )

    plotting_time = 0.
    start_time = timeit.default_timer()

    # go through training epochs
    for epoch in xrange(training_epochs):

        # go through the training set
        mean_cost = []
        for batch_index in xrange(n_train_batches):
            mean_cost += [train_rbm(batch_index)]

        print 'Training epoch %d, cost is ' % epoch, numpy.mean(mean_cost)

        # Plot filters after each training epoch
        plotting_start = timeit.default_timer()
        # Construct image from the weight matrix
        image = Image.fromarray(
            tile_raster_images(
                X=rbm.W.get_value(borrow=True).T,
                img_shape=(48, 48),
                tile_shape=(10, 10),
                tile_spacing=(1, 1)
            )
        )
        image.save('filters_at_epoch_%i.png' % epoch)
        plotting_stop = timeit.default_timer()
        plotting_time += (plotting_stop - plotting_start)

    end_time = timeit.default_timer()

    pretraining_time = (end_time - start_time) - plotting_time

    print ('Training took %f minutes' % (pretraining_time / 60.))
    # end-snippet-5 start-snippet-6
    #################################
    #     Sampling from the RBM     #
    #################################
    # find out the number of test samples
    number_of_test_samples = test_set_x.get_value(borrow=True).shape[0]

    # pick random test examples, with which to initialize the persistent chain
    test_idx = rng.randint(number_of_test_samples - n_chains)
    persistent_vis_chain = theano.shared(
        numpy.asarray(
            test_set_x.get_value(borrow=True)[test_idx:test_idx + n_chains],
            dtype=theano.config.floatX
        )
    )
    # end-snippet-6 start-snippet-7
    plot_every = 1000
    # define one step of Gibbs sampling (mf = mean-field) define a
    # function that does `plot_every` steps before returning the
    # sample for plotting
    (
        [
            presig_hids,
            hid_mfs,
            hid_samples,
            presig_vis,
            vis_mfs,
            vis_samples
        ],
        updates
    ) = theano.scan(
        rbm.gibbs_vhv,
        outputs_info=[None, None, None, None, None, persistent_vis_chain],
        n_steps=plot_every
    )

    # add to updates the shared variable that takes care of our persistent
    # chain :.
    updates.update({persistent_vis_chain: vis_samples[-1]})
    # construct the function that implements our persistent chain.
    # we generate the "mean field" activations for plotting and the actual
    # samples for reinitializing the state of our persistent chain
    sample_fn = theano.function(
        [],
        [
            vis_mfs[-1],
            vis_samples[-1]
        ],
        updates=updates,
        name='sample_fn'
    )

    # create a space to store the image for plotting ( we need to leave
    # room for the tile_spacing as well)
    image_data = numpy.zeros(
        (29 * n_samples + 1, 29 * n_chains - 1),
        dtype='uint8'
    )
    for idx in xrange(n_samples):
        # generate `plot_every` intermediate samples that we discard,
        # because successive samples in the chain are too correlated
        vis_mf, vis_sample = sample_fn()
        print ' ... plotting sample ', idx
        image_data[49 * idx:49 * idx + 48, :] = tile_raster_images(
            X=vis_mf,
            img_shape=(48, 48),
            tile_shape=(1, n_chains),
            tile_spacing=(1, 1)
        )

    # construct image
    image = Image.fromarray(image_data)
    image.save('samples.png')
    # end-snippet-7
    os.chdir('../')

In [5]:
test_rbm(dataset=datasets)

Shape.0
2050
Elemwise{mul,no_inplace}.0
Training epoch 0, cost is  -1.13252e+07
Training epoch 1, cost is  -3.61015e+07
Training epoch 2, cost is  -6.01769e+07
Training epoch 3, cost is  -8.87028e+07
Training epoch 4, cost is  -1.178e+08
Training epoch 5, cost is  -1.63161e+08
Training epoch 6, cost is  -1.8538e+08
Training epoch 7, cost is  -2.14662e+08
Training epoch 8, cost is  -2.55963e+08
Training epoch 9, cost is  -2.45663e+08
Training took 10.696327 minutes
 ... plotting sample  0


ValueError: could not broadcast input array from shape (48,685) into shape (48,405)